In [1]:
import torch
from torch import nn
from torch.nn import functional as f
import torchvision
import torchvision.transforms as transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 10
batch_size = 100
learning_rate = 0.001

# FashionMNIST dataset
train_dataset = torchvision.datasets.FashionMNIST(root='./data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.FashionMNIST(root='./data',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

class net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv2d_32 = nn.Conv2d(1,32,3,padding=1)
        self.conv2d_64 = nn.Conv2d(32,64,3,padding=1)
        self.max2d     = nn.MaxPool2d(2,2)
        self.conv2d_128 = nn.Conv2d(64,128,3,padding=1)
        self.conv2d_256 = nn.Conv2d(128,256,3, stride = 2,padding=1)
        self.linear1    = nn.Linear(3*3*256, 256)
        self.linear2    = nn.Linear(256,64)
        self.linear3    = nn.Linear(64,10)
        self.batch2d1     = nn.BatchNorm2d(64)
        self.batch2d2    = nn.BatchNorm2d(256)
        self.batch1d     = nn.BatchNorm1d(64)
        self.drop      = nn.Dropout(p=0.3)
        self.flat      = nn.Flatten()

    def forward(self,x):
            x = x.view(-1,1,28,28)
            x = f.relu(self.conv2d_32(x))
            x = f.relu(self.conv2d_64(x))
            x = self.batch2d1(x)
            x = f.relu(self.max2d(x))
            x = self.drop(x)

            x = f.relu(self.conv2d_128(x))
            x = f.relu(self.conv2d_256(x))
            x = self.batch2d2(x)
            x = f.relu(self.max2d(x))
            x = self.drop(x)

            x = self.flat(x)
            x = f.relu(self.linear1(x))
            x = self.drop(x)
            x = f.relu(self.linear2(x))
            x = self.drop(x)
            x = self.batch1d(x)
            x = f.log_softmax(self.linear3(x), dim=1)
            return(x)

model = net().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

100%|██████████| 26421880/26421880 [00:03<00:00, 8456932.08it/s] 


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 145663.83it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 2663348.68it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 7755846.62it/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Epoch [1/10], Step [100/600], Loss: 0.5247
Epoch [1/10], Step [200/600], Loss: 0.5378
Epoch [1/10], Step [300/600], Loss: 0.3769
Epoch [1/10], Step [400/600], Loss: 0.5244
Epoch [1/10], Step [500/600], Loss: 0.3824
Epoch [1/10], Step [600/600], Loss: 0.4986
Epoch [2/10], Step [100/600], Loss: 0.3900
Epoch [2/10], Step [200/600], Loss: 0.2351
Epoch [2/10], Step [300/600], Loss: 0.4112
Epoch [2/10], Step [400/600], Loss: 0.2694
Epoch [2/10], Step [500/600], Loss: 0.2634
Epoch [2/10], Step [600/600], Loss: 0.3660
Epoch [3/10], Step [100/600], Loss: 0.2966
Epoch [3/10], Step [200/600], Loss: 0.2478
Epoch [3/10], Step [300/600], Loss: 0.3092
Epoch [3/10], Step [400/600], Loss: 0.2618
Epoch [3/10], Step [500/600], Loss: 0.1927
Epoch [3/10], Step [600/600], Loss: 0.3007
Epoch [4/10], Step [100/600], Loss: 0.1889
Epoch [4/10], Step [200/600], Loss: 0.2163
Epoch [4/10], Step [300/600], Loss: 0.1171
Epoch [4

Batch Normalization 클래스
- 학습 도중 하위 계층의 가중치가 변함에 따라 상위 계층이 받는 입력 데이터의 분포가 바뀌는 현상을 방지하기 위하여 각 계층의 입력을 정규화
- 배치 정규화는 딥 러닝 모델에서 매우 중요한 역할을 하는 요소로, 각 레이어의 입력을 정규화하여 학습을 더 안정적이고 빠르게 만드는 기술. - 이는 그래디언트 소실 문제를 완화하고, 일반적으로 더 빠른 훈련 속도와 더 나은 최종 성능을 가져오며, 규제 효과도 가진다.
- nn.BatchNorm2d(64)는  2차원 입력에 대한 배치 정규화를 수행하며  64개의 채널에 대해 배치 정규화를 적용
- 이 계층을 통과하면 각 채널의 데이터가 평균 0과 분산 1을 갖도록 정규화

In [3]:
from torch import nn # 신경망 모듈을 포함하는 서브패키지
from torch.nn import functional as f
import torchvision # 컴퓨터 비전용 데이터셋 모델 등을 포함하는 라이브러리
import torchvision.transforms as transforms # 데이터 전처리를 위한 변환 함수들을 포함하는 서브 패키지

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 10
batch_size = 100
learning_rate = 0.001

train_dataset = torchvision.datasets.FashionMNIST( root = './data',
                                                  train = True,
                                                   transform = transforms.ToTensor(),
                                                   download = True)

test_dataset = torchvision.datasets.FashionMNIST( root = './data',
                                                  train = False,
                                                   transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = False)

# 신경망 모델정의

class net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv2d_32 = nn.Conv2d(1,32,3,padding=1)
        self.conv2d_64 = nn.Conv2d(32,64,3,padding=1)
        self.max2d     = nn.MaxPool2d(2,2)
        self.conv2d_128 = nn.Conv2d(64,128,3,padding=1)
        self.conv2d_256 = nn.Conv2d(128,256,3, stride = 2,padding=1)
        self.linear1    = nn.Linear(3*3*256, 256)
        self.linear2    = nn.Linear(256,64)
        self.linear3    = nn.Linear(64,10)
        self.batch2d1     = nn.BatchNorm2d(64)
        self.batch2d2    = nn.BatchNorm2d(256)
        self.batch1d     = nn.BatchNorm1d(64)
        self.drop      = nn.Dropout(p=0.3)
        self.flat      = nn.Flatten()

    #  데이터가 모델을 통과할때의 순전파 연산을 정의

    def forward(self,x):
            x = x.view(-1,1,28,28)
            x = f.relu(self.conv2d_32(x))
            x = f.relu(self.conv2d_64(x))
            x = self.batch2d1(x)
            x = f.relu(self.max2d(x))
            x = self.drop(x)

            x = f.relu(self.conv2d_128(x))
            x = f.relu(self.conv2d_256(x))
            x = self.batch2d2(x)
            x = f.relu(self.max2d(x))
            x = self.drop(x)

            x = self.flat(x)
            x = f.relu(self.linear1(x))
            x = self.drop(x)
            x = f.relu(self.linear2(x))
            x = self.drop(x)
            x = self.batch1d(x)
            x = f.log_softmax(self.linear3(x), dim=1)
            return(x)

model = net().to(device) # 모델 인스턴스화 및 디바이스로 이동

# Loss and optimizer
criterion = nn.CrossEntropyLoss() # 분류작업에 대한 손실함수
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # 모델의 파라미터를 최적화

# Train the model
# 주어진 에폭 수에 대해 데이터 로더를 반복하면서 수전파 손실 계산 역전파 옵티마이저 업데이트를 수행
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# 테스트 데이터 로드를 사용하여 모델의 출력을 얻고 정확도를 계산
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1) # 각 샘플에 대해 가장 확률이 높은 클래스와 인덱스를 predicted에 저장 1은 확률을 나타내는 차원
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Epoch [1/10], Step [100/600], Loss: 0.8331
Epoch [1/10], Step [200/600], Loss: 0.5083
Epoch [1/10], Step [300/600], Loss: 0.6214
Epoch [1/10], Step [400/600], Loss: 0.5235
Epoch [1/10], Step [500/600], Loss: 0.2782
Epoch [1/10], Step [600/600], Loss: 0.3757
Epoch [2/10], Step [100/600], Loss: 0.3708
Epoch [2/10], Step [200/600], Loss: 0.3382
Epoch [2/10], Step [300/600], Loss: 0.3006
Epoch [2/10], Step [400/600], Loss: 0.2493
Epoch [2/10], Step [500/600], Loss: 0.3237
Epoch [2/10], Step [600/600], Loss: 0.2006
Epoch [3/10], Step [100/600], Loss: 0.3723
Epoch [3/10], Step [200/600], Loss: 0.2400
Epoch [3/10], Step [300/600], Loss: 0.2604
Epoch [3/10], Step [400/600], Loss: 0.2685
Epoch [3/10], Step [500/600], Loss: 0.1364
Epoch [3/10], Step [600/600], Loss: 0.2590
Epoch [4/10], Step [100/600], Loss: 0.1580
Epoch [4/10], Step [200/600], Loss: 0.3621
Epoch [4/10], Step [300/600], Loss: 0.1649
Epoch [4/10], Step [400/600], Loss: 0.3024
Epoch [4/10], Step [500/600], Loss: 0.2094
Epoch [4/10

keras 적용

In [8]:
import tensorflow as tf

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(train_images, train_labels), (test_images, test_labels) = fashion_mnist

In [10]:
print(train_images[0])
print(test_images[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0  13  73   0
    0   1   4   0   0   0   0   1   1   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3   0  36 136 127  62
   54   0   0   0   1   3   4   0   0   3]
 [  0   0   0   0   0   0   0   0   0   0   0   0   6   0 102 204 176 134
  144 123  23   0   0   0   0  12  10   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 155 236 207 178
  107 156 161 109  64  23  77 130  72  15]
 [  0   0   0   0   0   0   0   0   0   0   0   1   0  69 207 223 218 216
  216 163 127 121 122 146 141  88 172  66]
 [  0   0   0   0   0   0   0   0   0   1   1   1   0 200 232 23

In [11]:
 train_images, test_images = train_images / 255.0, test_images/ 255.0

In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = (28, 28)),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(10, activation = 'softmax')
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [18]:
model.fit(train_images, train_labels, epochs = 5)

Epoch 1/5
1875/1875 [==============================] - 6s 2ms/step - loss: 0.4760 - accuracy: 0.8281
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3608 - accuracy: 0.8686
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3200 - accuracy: 0.8814
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2985 - accuracy: 0.8890
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2799 - accuracy: 0.8968


In [19]:
loss, accuracy = model.evaluate(test_images, test_labels)
print(loss, accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 0.3425 - accuracy: 0.8783
0.3425152003765106 0.8783000111579895


https://github.com/ratsgo/ratsnlp/tree/master/ratsnlp/nlpbook

## 트랜스퍼 러닝
- 트랜스퍼 러닝(transfer learning)이란 특정 태스크를 학습한 모델을 다른 태스크 수행에 재사용하는 기법
- 트랜스퍼 러닝이 주목받게 된 것은 업스트림 태스크와 프리트레인 덕분. 자연어의 풍부한 문맥(context)을 모델에 내재화하고 이 모델을 다양한 다운스트림 태스크에 적용해 그 성능을 대폭 끌어올림
- 대표적인 업스트림 태스크 가운데 하나가 다음 단어 맞추기입니다. GPT 계열 모델이 바로 이 태스크로 프리트레인을 수행
- 다른 업스트림 태스크로는 빈칸 채우기가 있습니다. BERT 계열 모델이 바로 이 태스크로 프리트레인을 수행. ‘빈칸 채우기’로 업스트림 태스크를 수행한 모델을 마스크 언어 모델(Masked Language Model)
- 다음 단어 맞히기, 빈칸 채우기 같은 업스트림 태스크는 강력한 힘을 지니며 뉴스, 웹 문서, 백과사전 등 글만 있으면 수작업 없이도 다량의 학습 데이터를 아주 싼값에 만들어 낼 수 있다. 이처럼 데이터 내에서 정답을 만들고 이를 바탕으로 모델을 학습하는 방법을 자기 지도 학습(self-supervised learning)이라고 함.
- 다운스트림 태스크의 학습 방식은 모두 파인튜닝(fine-tuning)이며 프리트레인을 마친 모델을 다운스트림 태스크에 맞게 업데이트하는 기법

## 바이트 페어 인코딩(Byte Pair Encoding, BPE)
- 원래 정보를 압축하는 알고리즘으로 제안되었는데 최근에는 자연어 처리 모델에 널리 쓰이고 있는 토큰화 기법
- 데이터에서 가장 많이 등장한 문자열을 병합해서 데이터를 압축하는 기법

## 워드피스(wordpiece)
- 말뭉치에서 자주 등장한 문자열을 토큰으로 인식한다는 점에서 BPE와 본질적으로 유사. 다만 어휘 집합을 구축할 때 문자열을 병합하는 기준이 다다. 워드피스는 BPE처럼 단순히 빈도를 기준으로 병합하는 것이 아니라, 병합했을 때 말뭉치의 우도(likelihood)를 가장 높이는 쌍을 병합